# Luftdaten data : data cleaning, resampling - mini version - cleaning v3

# - RETRYING THIS, just to makes sure we've got the hang of it! 
## Code builds a continuous time tabular version of the luftdaen data, such that the same time period is present for each sensor in the data, regardless of whether each sensor has data for all the time slots. 

## Testing :
- using pd.resample
- constructing a time shift using pandas own tools, rather than my own


#### Reference documents

Resampling time series data with Pandas ( Ben Alex Keen ) 
http://benalexkeen.com/resampling-time-series-data-with-pandas/

Pandas reference manual : 

.at - access df values using nay kind of index, for SINGLE VALUES
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html#pandas.DataFrame.loc

.iat - only integer index values for getting/setting SINGLE df values
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iat.html

.loc - 
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html#pandas.DataFrame.loc

.iloc - purely integer indexed access ( getting/setting ) values 
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html#pandas.DataFrame.iloc

datetime - documentation - useful for time! 
https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html

#### methods of filling … 

These are some of the common methods you might use for resampling:

Method	Description

bfill	Backward fill

count	Count of values

ffill	Forward fill

first	First valid data value

last	Last valid data value

max	Maximum data value

mean	Mean of values in time range

median	Median of values in time range

min	Minimum data value

nunique	Number of unique values

ohlc	Opening value, highest value, lowest value, closing value

pad	Same as forward fill

std	Standard deviation of values

sum	Sum of values

var	Variance of values

#### time abbreviations 

Alias	Description

B	Business day

D	Calendar day

W	Weekly

M	Month end

Q	Quarter end

A	Year end

BA	Business year end

AS	Year start

H	Hourly frequency

T, min	Minutely frequency

S	Secondly frequency

L, ms	Millisecond frequency

U, us	Microsecond frequency

N, ns	Nanosecond frequency

In [68]:
import pandas as pd
import numpy as np
import time

In [69]:
# parameters

# start_time = "2018-12-31 21:58:42"
end_time = "2019-01-01 11:58:42"
# generate this please
start_time = "?????"

time_frequency_for_periods__for_basic_data = "5T"
num_of_time_periods___for_basic_data = 24*20 # 24 hrs * 12 x 5 mins in each hour

# when generating time periods 
sampling_frequency = "3T"



# --- data urls 

curr_url = "????"
nordic_midnight_24_hrs_data__url = "/Users/miska/Documents/open_something/luftdaten/luftdaten_code/luftdaten__make_tabular_data__from_db_data/ld_NYE_midnight_24hrs_nordics_all_data_01.csv"
# nordic_midnight_24_hrs_data__url = "/home/miska/documents/opensomething/luftdaten/dustmin_to_csv__various_code/ld_NYE_midnight_24hrs_nordics_all_data_01.csv"



# set the current data source 
curr_url = nordic_midnight_24_hrs_data__url

In [70]:
# try convert the timestamp in the data, to epoch

in_data = pd.read_csv( curr_url )
in_data.shape

(127109, 7)

#### basic data checking

In [71]:
in_data.dtypes

sensor_id         int64
sensor_namee     object
lat             float64
lon             float64
timestamp        object
p1              float64
p2              float64
dtype: object

In [72]:
# is the timestamp column not an offical timestamp column?
type( in_data['timestamp'][0] )

str

In [73]:
# aha - timestamp column not a timestamp column?
# - let's fix 
in_data['timestamp'] = pd.to_datetime( in_data['timestamp'] )

In [74]:
# check the timestamps column type again
type( in_data['timestamp'][0] )

pandas._libs.tslibs.timestamps.Timestamp

In [75]:
# set the timestamp column as the index 
in_data = in_data.set_index( 'timestamp' )

In [76]:
in_data.index

DatetimeIndex(['2018-12-31 11:57:22', '2018-12-31 11:58:44',
               '2018-12-31 11:58:47', '2018-12-31 11:56:41',
               '2018-12-31 11:57:42', '2018-12-31 11:57:52',
               '2018-12-31 11:58:51', '2018-12-31 11:58:28',
               '2018-12-31 11:57:18', '2018-12-31 11:57:22',
               ...
               '2019-01-01 11:59:41', '2019-01-01 11:59:46',
               '2019-01-01 11:57:19', '2019-01-01 11:59:59',
               '2019-01-01 11:56:55', '2019-01-01 11:58:57',
               '2019-01-01 11:59:36', '2019-01-01 11:59:41',
               '2019-01-01 11:57:12', '2019-01-01 11:58:42'],
              dtype='datetime64[ns]', name='timestamp', length=127109, freq=None)

In [77]:
in_data = in_data.sort_index()

In [78]:
# check
in_data.index

DatetimeIndex(['2018-12-31 11:55:19', '2018-12-31 11:56:37',
               '2018-12-31 11:56:38', '2018-12-31 11:56:39',
               '2018-12-31 11:56:40', '2018-12-31 11:56:40',
               '2018-12-31 11:56:40', '2018-12-31 11:56:41',
               '2018-12-31 11:56:42', '2018-12-31 11:56:43',
               ...
               '2019-01-01 23:59:54', '2019-01-01 23:59:54',
               '2019-01-01 23:59:55', '2019-01-01 23:59:55',
               '2019-01-01 23:59:56', '2019-01-01 23:59:56',
               '2019-01-01 23:59:56', '2019-01-01 23:59:56',
               '2019-01-01 23:59:57', '2019-01-01 23:59:58'],
              dtype='datetime64[ns]', name='timestamp', length=127109, freq=None)

In [79]:
# order by time? 
in_data[:20]

,sensor_id,sensor_namee,lat,lon,p1,p2
timestamp,,,,,,
2018-12-31 11:55:19,13012,SDS011,57.662,12.006,6.30,2.60
2018-12-31 11:56:37,18112,SDS011,57.478,11.978,174.80,15.13
2018-12-31 11:56:38,15067,SDS011,60.024,18.770,1.62,1.02
2018-12-31 11:56:39,11765,SDS011,55.716,13.244,33.95,13.40
2018-12-31 11:56:40,14811,SDS011,57.706,11.900,63.25,10.33
2018-12-31 11:56:40,10827,SDS011,59.334,13.512,11.45,6.00
2018-12-31 11:56:40,17538,SDS011,55.612,12.972,13.68,3.12
2018-12-31 11:56:41,7406,SDS011,56.964,24.128,11.05,6.62
2018-12-31 11:56:42,16155,SDS011,59.832,17.632,1.66,1.10


In [80]:
# this works :) 
# in_data = in_data.sort_index()

In [81]:
# let's try sorting the index in a different way … just for the sake of trying
in_data = in_data.sort_values( by='timestamp' )

#### quick data printout

In [82]:
in_data.head()

,sensor_id,sensor_namee,lat,lon,p1,p2
timestamp,,,,,,
2018-12-31 11:55:19,13012,SDS011,57.662,12.006,6.30,2.60
2018-12-31 11:56:37,18112,SDS011,57.478,11.978,174.80,15.13
2018-12-31 11:56:38,15067,SDS011,60.024,18.770,1.62,1.02
2018-12-31 11:56:39,11765,SDS011,55.716,13.244,33.95,13.40
2018-12-31 11:56:40,14811,SDS011,57.706,11.900,63.25,10.33


In [83]:
in_data.tail()

,sensor_id,sensor_namee,lat,lon,p1,p2
timestamp,,,,,,
2019-01-01 23:59:56,12129,SDS011,59.340,18.040,1.85,0.80
2019-01-01 23:59:56,17538,SDS011,55.612,12.972,6.08,1.53
2019-01-01 23:59:56,12131,SDS011,59.450,17.916,1.48,0.52
2019-01-01 23:59:57,10843,SDS011,59.354,18.364,7.26,1.60
2019-01-01 23:59:58,11374,SDS011,59.258,18.008,2.65,2.01


#### quick data exploration 

In [84]:
start_time = pd.to_datetime( '2018-12-31 12:00:00' )
start_time

Timestamp('2018-12-31 12:00:00')

In [85]:
startime_plus_five_mins = start_time + pd.offsets.Minute( 5 )
startime_plus_five_mins

Timestamp('2018-12-31 12:05:00')

In [86]:
end_time = pd.to_datetime( '2019-01-01 12:00:00' )
end_time

Timestamp('2019-01-01 12:00:00')

In [87]:
end_time_minus_five_mins = end_time - pd.offsets.Minute(5 )
end_time_minus_five_mins

Timestamp('2019-01-01 11:55:00')

In [88]:
in_data_start_time = in_data[:1]

In [89]:
type( in_data_start_time )

pandas.core.frame.DataFrame

In [90]:
type( in_data_start_time.index ) #iat[ 0, ]

pandas.core.indexes.datetimes.DatetimeIndex

In [91]:
in_data_start_time.index

DatetimeIndex(['2018-12-31 11:55:19'], dtype='datetime64[ns]', name='timestamp', freq=None)

In [92]:
in_data_start_time.set_index( pd.DatetimeIndex( [ start_time ] )   )

,sensor_id,sensor_namee,lat,lon,p1,p2
2018-12-31 12:00:00,13012,SDS011,57.662,12.006,6.3,2.6


In [93]:
in_data_start_time

,sensor_id,sensor_namee,lat,lon,p1,p2
timestamp,,,,,,
2018-12-31 11:55:19,13012,SDS011,57.662,12.006,6.3,2.6


# try making start and end time rows from this table, in the hope it'll work better… 

In [94]:
start_time_dataframe_row = given_sensors_data_MINIMAL.iloc[:1]

In [95]:
start_time_dataframe_row.shape

(1, 2)

In [96]:
start_time_dataframe_row

,p1,p2
timestamp,,
2018-12-31 11:55:19,NaN,NaN


In [97]:
type( start_time_dataframe_row ) 

pandas.core.frame.DataFrame

In [98]:
start_time_dataframe_row.index = pd.DatetimeIndex( [start_time] )

In [99]:
start_time_dataframe_row

,p1,p2
2018-12-31 12:00:00,NaN,NaN


In [100]:
out = start_time_dataframe_row.append( given_sensors_data_MINIMAL )

In [101]:
out.head()

,p1,p2
2018-12-31 12:00:00,NaN,NaN
2018-12-31 11:55:19,NaN,NaN
2018-12-31 12:00:55,4.23,2.70
2018-12-31 12:03:25,4.98,2.66
2018-12-31 12:05:55,7.73,2.85


#### now make a blank end time… 

In [102]:
end_time_dataframe_row = given_sensors_data_MINIMAL.iloc[:1]

In [103]:
end_time_dataframe_row.index = pd.DatetimeIndex( [ end_time ] )

In [104]:
end_time_dataframe_row

,p1,p2
2019-01-01 12:00:00,NaN,NaN


In [105]:
out = out.append( end_time_dataframe_row )

In [106]:
out.head()

,p1,p2
2018-12-31 12:00:00,NaN,NaN
2018-12-31 11:55:19,NaN,NaN
2018-12-31 12:00:55,4.23,2.70
2018-12-31 12:03:25,4.98,2.66
2018-12-31 12:05:55,7.73,2.85


In [107]:
out.tail()

,p1,p2
2019-01-01 23:48:07,1.28,0.56
2019-01-01 23:50:37,1.36,0.65
2019-01-01 23:55:37,1.46,0.50
2019-01-01 23:58:07,1.12,0.46
2019-01-01 12:00:00,NaN,NaN


#### now try do the data interpolation, for each sensor, so there's the same time period in each sensor

In [108]:
##### work on a smaller section of the data
in_data__smlr = in_data[:]
in_data__smlr.shape

(127109, 6)

In [109]:
# check how it looks for how many sensors 
first_five_mins_rows__SMLR = in_data__smlr[ '2019-01-01 12:00:00' : '2019-01-01 12:05:00' ]
first_five_mins_rows__SMLR.shape, first_five_mins_rows__SMLR['sensor_id'].unique().shape

((314, 6), (178,))

In [110]:
# - for timing 

start_time = time.time()

# - the rest

# let's cheat a bit and output things in a list
out_time_complete_arrays = []
out_time_complete_arrays__WITHOUT_WHERE_CLEANING = []

list_of_unique_sensor_ids = in_data__smlr['sensor_id'].unique()

## print("- got "+str( list_of_unique_sensor_ids.shape )+" unique sensor ids \n")

length_of_list_of_unique_sensor_ids = len( list_of_unique_sensor_ids )

for curr_sensor_id in list_of_unique_sensor_ids[:]:
    
    print("\n --  working on sensor id "+str( curr_sensor_id )+"/"+str(length_of_list_of_unique_sensor_ids) )
    
    # -- fetch all rows for a given sensor 
    curr_sensor_data = in_data__smlr[ in_data__smlr['sensor_id'] == curr_sensor_id  ]
    
    ### print("-- -- and that has "+str( curr_sensor_data.shape )+" entries " )
    
    ### print("-- -- - the start date row looks like this ")
    ### print( start_time_dataframe_row )
    
    # generate a minimised version of the data such that one only has the time index, p1 and p2 columns
    curr_sensor_data__minimised =  curr_sensor_data[ ['p1', 'p2'] ]
    
    ### print( "-- -- - curr_sensor_data__minimised.index = "+str( curr_sensor_data__minimised.index ) )
    ### print( "-- -- - curr_sensor_data__minimised.columns = "+str( curr_sensor_data__minimised.columns ) )    
    
    ### print("-- -- - curr_sensor_data__minimised.shape = "+str( curr_sensor_data__minimised.shape ) )
    
    ### print(  curr_sensor_data__minimised.head()  )
    
    # -- now add beginning and end time data frames 
    
    ### print( "-- -- - start_time_dataframe_row.shape = "+str( start_time_dataframe_row.shape ) )
    ### print( "-- -- - start_time_dataframe_row.columns = "+str( start_time_dataframe_row.columns ) )
    
    curr_sensor_data_min_w_relv_start_n_end_times = start_time_dataframe_row.append( curr_sensor_data__minimised )
    
    # then add the end time
    
    curr_sensor_data_min_w_relv_start_n_end_times = curr_sensor_data_min_w_relv_start_n_end_times.append( end_time_dataframe_row )
    
    ## print("-- -- - curr_sensor_data_min_w_relv_start_n_end_times.shape = "+str(curr_sensor_data_min_w_relv_start_n_end_times.shape) )
    
    # -- NOW ASSEMBLE THE DATA! 
    
    ## print( curr_sensor_data_min_w_relv_start_n_end_times.head() )
    ## print( curr_sensor_data_min_w_relv_start_n_end_times.tail() )
    
    #
    
    ## print("-- -- - curr_sensor_data_min_w_relv_start_n_end_times.index = "+str( curr_sensor_data_min_w_relv_start_n_end_times.index ) )
    
    # -- now resample 
    
    array_resampled = curr_sensor_data_min_w_relv_start_n_end_times.resample("5Min").mean()
    
    ### print("-- -- - array_resampled : "+str( array_resampled ) )
    
    # -- fill the NaNs … 
    
    # SAVE A COPY WIHTOUT WHERE CLEANING 
    out_time_complete_arrays__WITHOUT_WHERE_CLEANING.append( array_resampled )
    
    # array_resampled['p1'] = array_resampled['p1'].where( array_resampled['p1'] == np.NaN, 0 )
    ### array_resampled['p2'] = array_resampled.where( array_resampled['p2'] == np.NaN, 0 )    
    
    # - clean 
    array_resampled = array_resampled.fillna( 0 )
    
    # -- export
    
    out_time_complete_arrays.append( array_resampled )
    
    
print("\n -- | -- | -- |  and all that took "+str( time.time() - start_time )+" seconds" )


 --  working on sensor id 13012/205

 --  working on sensor id 18112/205

 --  working on sensor id 15067/205

 --  working on sensor id 11765/205

 --  working on sensor id 14811/205

 --  working on sensor id 10827/205

 --  working on sensor id 17538/205

 --  working on sensor id 7406/205

 --  working on sensor id 16155/205

 --  working on sensor id 11058/205

 --  working on sensor id 16533/205

 --  working on sensor id 14807/205

 --  working on sensor id 10924/205

 --  working on sensor id 10723/205

 --  working on sensor id 17923/205

 --  working on sensor id 14294/205

 --  working on sensor id 17662/205

 --  working on sensor id 18838/205

 --  working on sensor id 13419/205

 --  working on sensor id 12389/205

 --  working on sensor id 14264/205

 --  working on sensor id 16296/205

 --  working on sensor id 16151/205

 --  working on sensor id 19207/205

 --  working on sensor id 11404/205

 --  working on sensor id 12387/205

 --  working on sensor id 18110/205

 

In [115]:
out_time_complete_arrays[5]

,p1,p2
2018-12-31 11:55:00,11.450000,6.000000
2018-12-31 12:00:00,11.610000,6.110000
2018-12-31 12:05:00,10.775000,6.185000
2018-12-31 12:10:00,13.000000,6.150000
2018-12-31 12:15:00,16.395000,8.180000
2018-12-31 12:20:00,18.637500,9.150000
2018-12-31 12:25:00,0.000000,0.000000
2018-12-31 12:30:00,15.870000,8.330000
2018-12-31 12:35:00,21.980000,9.500000
2018-12-31 12:40:00,24.120000,9.460000
